At first we load the data and do the pre-processing on it 
1. check if there is any NULL in the dataset 
2. normalize the dataset with z-score and transfer the dataset to the new space with the mean of zero and the standard deviation of 1

In [4]:
import pandas as pd
data_labeled_train = pd.read_csv('labeled_train_set.csv')
data_labeled_test = pd.read_csv('test_set.csv')
data_unlabeled_train = pd.read_csv('unlabeled_train_set.csv')